2)	At Supplier or Manufacturer level report on the growth in terms of total revenue. 
    It can also be broken By procedure, By category which supplier has grown more.

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# reading the data
data = pd.read_excel('complete_data.xlsx', index_col=0)

In [ ]:
# getting an overview of our data
print("Our data has {0} rows and {1} columns".format(data.shape[0], data.shape[1]))
# checking for missing values
print("Are there missing values? {}".format(data.isnull().any().any()))
data.describe()

In [ ]:
data['Month']= pd.DatetimeIndex(data['com_procedure_date']).month

In [ ]:
pivot_table=pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category','Month'],columns='com_supplier', values=["com_cost_price"],aggfunc={'com_cost_price': np.mean}).apply(np.ceil)

In [ ]:
Month_wise_total_expen=pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category','Month'],values=["com_cost_price"],aggfunc={'com_cost_price': np.sum}).apply(np.ceil)

In [ ]:
Month_wise_total_expen = pd.DataFrame(Month_wise_total_expen.to_records())

In [ ]:
pt_tot_expend = pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category','Month'],columns='com_supplier', values=["com_cost_price","com_quantity_consumed"],aggfunc={'com_cost_price': np.mean,'com_quantity_consumed':np.sum})

In [ ]:
def cheapest_supplier_table(pt_tot_expend,Month_wise_total_expen):
    flat_table=pd.DataFrame(pt_tot_expend.to_records())
    Result = pd.DataFrame(columns=['Procedure','Main_Category','1','2','3','4','5','6','7'])
    for procedure in (flat_table['com_procedurename'].unique()).tolist():
        main_category = (flat_table.loc[flat_table['com_procedurename']== procedure]['com_bom_main_category'].unique()).tolist()
        for category in main_category:
            Main_dict={}
            months=(flat_table.loc[(flat_table['com_procedurename']== procedure)&(flat_table['com_bom_main_category']==category)]['Month']).tolist()
            for month in months:
                extracted_cost_price=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category) &(Month == @month)').dropna(axis=1).loc[:,('com_cost_price',slice(None))]
                extracted_qty=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category) &(Month == @month)').dropna(axis=1).loc[:,'com_quantity_consumed']
                Cheapest_supplier_name=list(extracted_cost_price.idxmin(axis=1).values[0])[1]
                Cheapest_supp_qty=extracted_qty[Cheapest_supplier_name].values[0]
                Cheapest_cost_price=list(extracted_cost_price.min(axis=1))[0]
                Cheapest_supp_expend=Cheapest_cost_price*Cheapest_supp_qty
                Total_expend=Month_wise_total_expen[(Month_wise_total_expen['com_procedurename']==procedure)&(Month_wise_total_expen['com_bom_main_category']==category)&(Month_wise_total_expen['Month']==month)]['com_cost_price'].tolist()[0]
                sub_dict={'Supplier_Name':Cheapest_supplier_name,'Qty':Cheapest_supp_qty,'Cost_Price':Cheapest_cost_price,'Cheapest_supp_expend':Cheapest_supp_expend,'Total Expend': Total_expend}
                Main_dict.update({month:sub_dict})
            Result= Result.append({'Procedure':procedure,'Main_Category':category,
                                   '1':Main_dict.get(1),'2':Main_dict.get(2),'3':Main_dict.get(3),
                                   '4':Main_dict.get(4),'5':Main_dict.get(5),'6':Main_dict.get(6),
                                   '7':Main_dict.get(7)},ignore_index=True)
        
    return Result        
        

In [ ]:
rs=cheapest_supplier_table(pt_tot_expend, Month_wise_total_expen)

In [ ]:
table_of_cheap_supp=rs

In [ ]:
def cheapest_supplier_trend(table_of_cheap_supp,month1,month2,month3):
    Result = pd.DataFrame(columns=['Procedure','Main_Category','Is_curr_%_of_CS_expend_more'])
    for procedure in (table_of_cheap_supp['Procedure'].unique()).tolist():
        main_category = (table_of_cheap_supp.loc[table_of_cheap_supp['Procedure']== procedure]['Main_Category']).tolist()
        for category in main_category:
            row_data=(table_of_cheap_supp.loc[(table_of_cheap_supp['Procedure']==procedure)&(table_of_cheap_supp['Main_Category']==category)])
            row_data.set_index(['Procedure', 'Main_Category'],inplace=True)
            past=row_data.loc[:, str(month1):str(month2)]
            current=row_data[str(month3)][procedure][category]
            total_cheap_sup_exp=0.0
            tot_exp=0.0
            for i in range(month1,month2+1):
                if (past[str(i)][procedure][category]) is None:
                    continue
                else:
                    total_cheap_sup_exp +=past[str(i)][procedure][category].get('Cheapest_supp_expend')
                    tot_exp +=past[str(i)][procedure][category].get('Total Expend')
            
            if (tot_exp==0.0):
                past_cheap_supp_avg_amt=0.0
            else:
                past_cheap_supp_avg_amt=(total_cheap_sup_exp/tot_exp)*100
            
            if current is None:
                curr_cheap_supp_avg_amt=0.0
            else:
                curr_cheap_supp_avg_amt=(current.get('Cheapest_supp_expend')/current.get('Total Expend'))*100
            
            if(curr_cheap_supp_avg_amt>past_cheap_supp_avg_amt):
                Is_CS_expend_more=1
            else:
                Is_CS_expend_more=0
                
            Result= Result.append({'Procedure':procedure,'Main_Category':category,'Is_curr_%_of_CS_expend_more':Is_CS_expend_more},ignore_index=True)
    return Result
                
    
    

In [ ]:
check= cheapest_supplier_trend(rs,2,5,6)

This is for only new month data
Check if the supplier who has cheapest average price per unit for that procedure, Main category combination has got the 80% of the total spend. If not flag it , because it tells us that we are spending on costlier suppliers instead of cheap suppliers.

In [ ]:
July_data=data[data['Month']==7]

In [ ]:
total_expend=pd.pivot_table(July_data, index = ['com_procedurename','com_bom_main_category'],values=["com_cost_price"],aggfunc={'com_cost_price': np.sum}).apply(np.ceil)

In [ ]:
total_expend = pd.DataFrame(total_expend.to_records())

In [ ]:
total_expend.rename(columns={'com_procedurename':'Procedure','com_bom_main_category':'Main_Category','com_cost_price':'Total_expenditure'},inplace=True)

In [ ]:
Low_cost_supp=pd.pivot_table(July_data, index = ['com_procedurename','com_bom_main_category'],columns='com_supplier', values=["com_cost_price","com_quantity_consumed"],aggfunc={'com_cost_price': np.mean ,'com_quantity_consumed':np.sum}).apply(np.ceil)

In [ ]:
def curr_month_low_cost_supp(pt_tot_expend):
    flat_table=pd.DataFrame(pt_tot_expend.to_records())
    Result = pd.DataFrame(columns=['Procedure','Main_Category','LC_supp_name','LC_supp_cost_price','Lc_supp_qty'])
    for procedure in (flat_table['com_procedurename'].unique()).tolist():
        main_category = (flat_table.loc[flat_table['com_procedurename']== procedure]['com_bom_main_category'].unique()).tolist()
        for category in main_category:
                extracted_cost_price=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category)').dropna(axis=1).loc[:,('com_cost_price',slice(None))]
                extracted_qty=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category)').dropna(axis=1).loc[:,'com_quantity_consumed']
                Cheapest_supplier_name=list(extracted_cost_price.idxmin(axis=1).values[0])[1]
                Cheapest_supp_qty=extracted_qty[Cheapest_supplier_name].values[0]
                Cheapest_cost_price=list(extracted_cost_price.min(axis=1))[0]
                Result= Result.append({'Procedure':procedure,'Main_Category':category,'LC_supp_name':Cheapest_supplier_name,
                                   'LC_supp_cost_price':Cheapest_cost_price,'Lc_supp_qty':Cheapest_supp_qty},ignore_index=True)    
    return Result        
        

In [ ]:
low_cost_supp_details=curr_month_low_cost_supp(Low_cost_supp)

In [ ]:
Merged=pd.merge(low_cost_supp_details,total_expend, on=['Procedure','Main_Category'])

In [ ]:
Merged['%_of_tot_expen']=((Merged['LC_supp_cost_price']*Merged['Lc_supp_qty'])/Merged['Total_expenditure'])

In [ ]:
Merged.to_excel("Merged.xlsx")

In [ ]:
to_check= Merged[Merged['%_of_tot_expen']<0.80]

In [1]:
import numpy as np

In [2]:
a=[1,2,-3,5,6]